Librerías

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
import os
import matplotlib.ticker as ti
from itertools import islice
from scipy.special import wofz
from scipy.optimize import curve_fit

**Actividad 1**

Importación de datos

In [16]:
df = pd.read_csv("actividad 1.csv", delimiter=';', skiprows=2)
df.dropna(how='any', inplace=True)
df.columns = ["theta_deg", "intensity"]
df["theta_deg"] = df["theta_deg"].str.replace(",", ".").astype(float)
df["intensity"] = df["intensity"].astype(float)
df.head()


,theta_deg,intensity
0,15.0,389.0
1,15.1,384.0
2,15.2,389.0
3,15.3,370.0
4,15.4,387.0


Ajuste

In [17]:
# Datos
x = df["theta_deg"].values
y = df["intensity"].values
y_err = np.sqrt(y)  # Error Poissoniano

# Funciones de ajuste
def lorentzian(x,A,x0,gamma):
    return A / (1 + ((x-x0)/gamma)**2)

def voigt_profile(x, A, x0, sigma, gamma):
    z = (x - x0 + 1j * gamma) / (sigma * np.sqrt(2))
    return A * np.real(wofz(z))

#Este x es en angulo
xv = [i[0] for i in data if 41.7 <= i[0] <= 42.1]
xl = [i[0] for i in data if 41.5 <= i[0] <= 42.2]

#Nuevo x en longitud de onda
xn = [2*d*np.sin(np.radians(k)) for k in xv]
xnl = [2*d*np.sin(np.radians(k)) for k in xl]

y = [i[1] for i in data if 41.7 <= i[0] <= 42.1 ]
yl = [i[1] for i in data if 41.5 <= i[0] <= 42.2]

xerr = np.full(len(xv),0.1)

xerrn = [np.abs(d*np.cos(np.radians(k))*np.radians(0.1)) for k in xv]
xerrnl = [np.abs(d*np.cos(np.radians(k))*np.radians(0.1)) for k in xl]

yerr = np.full(len(y),1)
yerrl = np.full(len(yl),1)

guess = [max(yl),2.68e-10,(max(xnl)-min(xnl))/10]

#lorentzian

param, cov = curve_fit(lorentzian,xnl,yl,p0=guess,sigma=yerrl,absolute_sigma=True)

covn = np.sqrt(np.diagonal(cov))

x_ = np.linspace(xnl[0]-xerrnl[0],xnl[-1]+xerrnl[-1],200)
y_ = lorentzian(x_, param[0], param[1], param[2])

#Voigt profile

guessv = [max(y),2.68e-10,(max(xn)-min(xn))/10,(max(xn)-min(xn))/20]
guessv2 = [param[0], param[1], 1, param[2]*20]
guessv3 = [1,2.68e-10, (max(xn)-min(xn)),(max(xn)-min(xn))]

paramv, covv = curve_fit(voigt_profile,xn,y,p0=guessv2, sigma=yerr, absolute_sigma=True)

covvn = np.sqrt(np.diagonal(covv))

xv_ = np.linspace(xn[0]-xerrn[0],xn[-1]+xerrn[-1],200)
yv_ = voigt_profile(x_,*paramv)

print(param[1])

fig, axs = plt.subplots(2, 1, figsize=(15,7.5), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0, 'wspace': 0.15})

fig.suptitle('Intensidad vs Longitud de onda', fontsize=20)

#axs[0,2].scatter(xn,y)

#Ajuste lorentziano

x0 = param[1]
error = covn[1]

exponente = np.floor(np.log10(x0))  # Encuentra el exponente más grande
x0_mant = x0 / 10**exponente
error_mant = error / 10**exponente

axs[0].errorbar(xnl, yl, xerr=xerrnl, yerr=yerrl, fmt='o', label='Pico #3', capsize=3)
axs[0].axvline(param[1],color='g', linestyle='--', label=f"$x_0 = ({x0_mant:.5f} \\pm {error_mant:.5f}) \\times 10^{{{int(exponente)}}}$")
axs[0].yaxis.set_major_formatter(ti.ScalarFormatter(useMathText=True))
axs[0].ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
axs[0].yaxis.get_offset_text().set_fontsize(8) 
axs[0].plot(x_, y_, label='Ajuste Lonrentziano', color='red')
axs[0].set_ylabel('Intensidad (imp/s)')
axs[0].legend(fontsize=8)

res = yl - lorentzian(xnl,*param) / yerrl

axs[1].scatter(xnl, res, color='black', marker='x')
axs[1].axhline(0, color='black', linestyle='--')
axs[1].yaxis.set_major_formatter(ti.ScalarFormatter(useMathText=True))
axs[1].ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
axs[1].yaxis.get_offset_text().set_fontsize(8)
axs[1].set_xlabel('Longitud de onda (m)')
axs[1].set_ylabel('Residuales\n normalizados')

NameError: name 'data' is not defined